In [101]:
import re

Пример DDL

In [ ]:
ddl = """ 
CREATE TABLE AuthAttempts(
	Id uniqueidentifier NOT NULL,
	CreatedOn datetime2(7) NOT NULL,
	MobilePhone nvarchar(21) NOT NULL,
	CurrentAttempt int NOT NULL,
	EndDate datetime2(7) NULL
)
"""

Получим название таблицы из DDL

In [103]:
table_name_pattern = re.compile(r'CREATE\s+TABLE\s+(\[?\w+\]?\.)?(\[?\w+\]?)', re.IGNORECASE)
match = table_name_pattern.search(ddl)

if match:
    table_name_raw = match.group(2).strip('[]')
    table_name = re.sub(r'([a-z])([A-Z])', r'\1_\2', table_name_raw.split('.')[-1]).lower()
    # table_name = re.sub(r'([a-z])([A-Z])', r'\1_\2', match.group(2).strip('[]').lower())
    print(f"Table name: {table_name}")
else:
    print("Table name not found.")

Table name: approval_code


Распознаем список колонок с типами данных

In [104]:
column_pattern = re.compile(r'\s*([\[\]A-Za-z0-9_]+)\s+([A-Za-z0-9_]+)(\([0-9, ]+\))?', re.IGNORECASE)

columns = {}
inside_table = False

for line in ddl.splitlines():
    line = line.strip()
    if line.upper().startswith('CREATE TABLE'):
        continue
    if line.startswith(')'):
        break
    match = column_pattern.match(line)
    if match:
        column_name = match.group(1)
        data_type = match.group(2).upper()
        columns[column_name] = data_type

columns

{'Id': 'UNIQUEIDENTIFIER',
 'CreatedOn': 'DATETIME2',
 'CreatedById': 'UNIQUEIDENTIFIER',
 'ModifiedOn': 'DATETIME2',
 'ModifiedById': 'UNIQUEIDENTIFIER',
 'ProcessListeners': 'INT',
 'TypeId': 'UNIQUEIDENTIFIER',
 'ExpiresOn': 'DATETIME2',
 'Source': 'NVARCHAR',
 'Code': 'NVARCHAR',
 'DeliveryAddress': 'NVARCHAR',
 'IncorrectEntersNumber': 'INT',
 'ContactId': 'UNIQUEIDENTIFIER',
 'WasUsed': 'BIT',
 'CurrentResendNumber': 'INT'}

Сделаем соответствие типов данных в MSSQL и Greenplum

In [105]:
mssql_to_greenplum = {
    # Числовые типы
    'BIGINT': 'BIGINT',
    'INT': 'INTEGER',
    'SMALLINT': 'SMALLINT',
    'TINYINT': 'SMALLINT',
    'BIT': 'BOOLEAN',
    'DECIMAL': 'NUMERIC',
    'NUMERIC': 'NUMERIC',
    'MONEY': 'NUMERIC(19,4)',
    'SMALLMONEY': 'NUMERIC(10,4)',
    'FLOAT': 'DOUBLE PRECISION',
    'REAL': 'REAL',

    # Строковые типы
    'VARCHAR': 'VARCHAR',
    'NVARCHAR': 'TEXT',
    'CHAR': 'CHAR',
    'NCHAR': 'CHAR',
    'TEXT': 'TEXT',
    'NTEXT': 'TEXT',

    # Дата и время
    'DATETIME': 'TIMESTAMP',
    'DATETIME2': 'TIMESTAMP',
    'SMALLDATETIME': 'TIMESTAMP',
    'DATE': 'DATE',
    'TIME': 'TIME',
    'DATETIMEOFFSET': 'TIMESTAMP WITH TIME ZONE',

    # Двоичные
    'BINARY': 'BYTEA',
    'VARBINARY': 'BYTEA',
    'IMAGE': 'BYTEA',

    # Другие
    'UNIQUEIDENTIFIER': 'UUID',
    'XML': 'TEXT', # ?
    'SQL_VARIANT': 'TEXT',  # В GP можно хранить как JSON/текст

    # Неизвестные/нестандартные
    'ROWVERSION': 'BYTEA',
    'TIMESTAMP': 'BYTEA'  # В MSSQL это не время, а бинарная метка
}


In [106]:
def map_mssql_to_greenplum(columns_dict, mapping_dict):
    mapped_columns = {}
    for col, mssql_type in columns_dict.items():
        gp_type = mapping_dict.get(mssql_type.upper(), 'TEXT')
        mapped_columns[col] = gp_type
    return mapped_columns

Получим список колонок в Greenplum с типами данных

In [107]:
greenplum_columns = map_mssql_to_greenplum(columns, mssql_to_greenplum)
greenplum_columns

{'Id': 'UUID',
 'CreatedOn': 'TIMESTAMP',
 'CreatedById': 'UUID',
 'ModifiedOn': 'TIMESTAMP',
 'ModifiedById': 'UUID',
 'ProcessListeners': 'INTEGER',
 'TypeId': 'UUID',
 'ExpiresOn': 'TIMESTAMP',
 'Source': 'TEXT',
 'Code': 'TEXT',
 'DeliveryAddress': 'TEXT',
 'IncorrectEntersNumber': 'INTEGER',
 'ContactId': 'UUID',
 'WasUsed': 'BOOLEAN',
 'CurrentResendNumber': 'INTEGER'}

In [108]:
## Добавим технические поля
greenplum_columns["EtlExecutionDate"] = "TIMESTAMP"
greenplum_columns["EtlStartDate"] = "TIMESTAMP"

Соберем схему

In [109]:
schema = [{"name": col, "type": dtype} for col, dtype in greenplum_columns.items()]
schema

[{'name': 'Id', 'type': 'UUID'},
 {'name': 'CreatedOn', 'type': 'TIMESTAMP'},
 {'name': 'CreatedById', 'type': 'UUID'},
 {'name': 'ModifiedOn', 'type': 'TIMESTAMP'},
 {'name': 'ModifiedById', 'type': 'UUID'},
 {'name': 'ProcessListeners', 'type': 'INTEGER'},
 {'name': 'TypeId', 'type': 'UUID'},
 {'name': 'ExpiresOn', 'type': 'TIMESTAMP'},
 {'name': 'Source', 'type': 'TEXT'},
 {'name': 'Code', 'type': 'TEXT'},
 {'name': 'DeliveryAddress', 'type': 'TEXT'},
 {'name': 'IncorrectEntersNumber', 'type': 'INTEGER'},
 {'name': 'ContactId', 'type': 'UUID'},
 {'name': 'WasUsed', 'type': 'BOOLEAN'},
 {'name': 'CurrentResendNumber', 'type': 'INTEGER'},
 {'name': 'EtlExecutionDate', 'type': 'TIMESTAMP'},
 {'name': 'EtlStartDate', 'type': 'TIMESTAMP'}]

Сохраним в файл

In [110]:
with open(f"tz/bpm/gp_schemas/{table_name}.py", "w+") as file:
    file.write(f"schema={schema}")